In [1]:
import pandas as pd
import numpy as np
import urllib.request as ureq
from bs4 import BeautifulSoup
import re

# Raw Scraper

In [2]:
# =====|| For Testing ||=====

# Define dictionaries to hold soup and path info

pathd_cat={}
soupd_cat={}
beg_path='https://www.platt.com'
cat_count=0
subcount=0
cat_names=[]
items=0

path='https://www.platt.com/platt-electric-supply/Bulbs-and-Ballasts/search.aspx?SectionID=10'
html=ureq.urlopen(path)

soup=BeautifulSoup(html,'html.parser')
type(soup)

linktable_cat=soup.find("div",{"class":"SubCategoryList"})
links_cat=linktable_cat.find_all('a')

df=pd.DataFrame() # define dataframe to append all dfsubs to

for link in links_cat:
       
    #print(link.get('href'))
    pathd_cat["path"+str(cat_count)]=link.get('href')
    tempath=beg_path+pathd_cat["path"+str(cat_count)]
    html=ureq.urlopen(tempath)


    soupd_cat["soup_cat"+str(cat_count)]=BeautifulSoup(html,'html.parser')
    type(soupd_cat["soup_cat"+str(cat_count)])
    title=soupd_cat["soup_cat"+str(cat_count)].title.get_text()
    print(">",title)
    
    pathd_sub={}
    soupd_sub={}
    subtitles=[]

    title_sub = soupd_cat["soup_cat"+str(cat_count)].title.get_text()
    if "Ballasts" not in title_sub:
        if "Drivers" not in title_sub:
            if "Fluorescent" in title_sub: # test statement so i don't have to wait ages
                linktable_sub=soupd_cat["soup_cat"+str(cat_count)].find("div",{"class":"SubCategoryList"})
                links_sub=linktable_sub.find_all('a')

                cat_count_sub=0
                dfsub=pd.DataFrame() # define dataframe to append all dfsubsub's to
                
                for link in links_sub:

                    pathd_sub["path"+str(subcount)]=link.get('href')
                    tempath=beg_path+pathd_sub["path"+str(subcount)]
                    html=ureq.urlopen(tempath)


                    soupd_sub["soup_cat"+str(subcount)]=BeautifulSoup(html,'html.parser')
                    type(soupd_sub["soup_cat"+str(subcount)])

                    subtitle=soupd_sub["soup_cat"+str(subcount)].title.get_text()
                    if "" in subtitle:
                        subtitles.append(subtitle)
                        print(">>",subtitle)

                        #subsubcategories
                        pathd_subsub={}
                        soupd_subsub={}
                        subsubcount=0
                        subsubtitles=[]

                        # if there is no descendent, skip it
                        linktable_subsub=soupd_sub["soup_cat"+str(subcount)].find("div",{"class":"SubCategoryList"})

                        try:
                            links_subsub=linktable_subsub.find_all('a')
                            cat_count_subsub=0
                            
                            dfsubsub=pd.DataFrame() # define dataframe to append all dfitems to

                            for link in links_subsub:

                                #print(link.get('href'))
                                pathd_subsub["path"+str(subsubcount)]=link.get('href')
                                tempath=beg_path+pathd_subsub["path"+str(subsubcount)]
                                html=ureq.urlopen(tempath)


                                soupd_subsub["soup_cat"+str(subsubcount)]=BeautifulSoup(html,'html.parser')
                                type(soupd_subsub["soup_cat"+str(subsubcount)])

                                subsubtitle=soupd_subsub["soup_cat"+str(subsubcount)].title.get_text()
                            
                                subsubtitles.append(subsubtitle)
                                print(">>>",subsubtitle)


                                #==================================================================================
                                pathd_pagination={}
                                soupd_pagination={}
                                page_count=1

                                pathd_item={}
                                soupd_item={}
                                item_count=0

                                # grab last link in pagination, and create format for page url population
                                linktable_pagination=soupd_subsub["soup_cat"+str(subsubcount)].find("div",{"class":"searchTopBarPagination"})
                                links_page=linktable_pagination.find_all('a')
                                lastlink=linktable_pagination.find('a',{"class":"pg-arrow-last"})

                                # account for pages where there may not be pagination
                                try:
                                    lastpage=lastlink.get('href')
                                    linkformat=beg_path+lastpage
                                    last_pagenum=linkformat[-6]

                                    # loop through the range of total pages and create urls, starting at 2 because we have already got the first page
                                    pagination_urls=list()

                                    for i in range(2,int(last_pagenum)+1):

                                        # replace page values
                                        url_new=linkformat[:int(-6)]+str(i)+linkformat[int(-6)+1:]
                                        pagination_urls.append(url_new)
                                    lock=0

                                except AttributeError:
                                    lock=1

                                # look at first item page
                                linktable_item=soupd_subsub["soup_cat"+str(subsubcount)].find("div",{"class":"SearchProductsTable"})
                                links_item=linktable_item.find_all('a',{"class":"productListCatNumLink"})
                                for link in links_item:
                                    pathd_item["path"+str(item_count)]=link.get('href')

                                    tempath=beg_path+pathd_item["path"+str(item_count)]
                                    html=ureq.urlopen(tempath)

                                    soupd_item["soup_cat"+str(item_count)]=BeautifulSoup(html,'html.parser')
                                    type(soupd_item["soup_cat"+str(item_count)])
                                    itemtitle=soupd_item["soup_cat"+str(item_count)].title.get_text()
                                    #print(testtitle)
                                    item_count+=1
                                    items+=1

                                print("There are",item_count,"items on page 1")
                                page1_cnt=item_count

                                pgitem_count=0    


                                # loop through remaining pages if there are any

                                if lock==0:
                                    for page in pagination_urls:

                                        # get soup for that page
                                        html=ureq.urlopen(page)
                                        soup=BeautifulSoup(html,'html.parser')
                                        type(soup)

                                        # grab item page links on pagination page
                                        linktable_item=soup.find("div",{"class":"SearchProductsTable"})
                                        links=linktable_item.find_all('a',{"class":"productListCatNumLink"})

                                        pgitem_count=0
                                        for link in links:
                                            pathd_item["path"+str(item_count)]=link.get('href')

                                            tempath=beg_path+pathd_item["path"+str(item_count)]
                                            html=ureq.urlopen(tempath)

                                            soupd_item["soup_cat"+str(item_count)]=BeautifulSoup(html,'html.parser')
                                            type(soupd_item["soup_cat"+str(item_count)])
                                            itemtitle=soupd_item["soup_cat"+str(item_count)].title.get_text()
                                            #print(itemtitle)
                                            item_count+=1
                                            pgitem_count+=1
                                        page_count+=1

                                        print("There are:",pgitem_count,"items on page",page_count)
                                print("There are",page_count,"page(s) in this descendent")
                                print("There are",item_count,"total items")
                                if lock==1:
                                    print("There is only one page")

                                # ===== DATAFRAME INFORMATION =====
                                # With soups for all item pages, iterate through soups to gather information, then create dataframe from them.

                                # define dataframe to hold item page info
                                dfitem=pd.DataFrame() #********ISSUE: DF REDEFINED EVERY LOOP
                                pg_count=0

                                # loop through item pages
                                for item in soupd_item:
                                    itemtitle=soupd_item['soup_cat'+str(pg_count)].title.get_text()
                                    #print(itemtitle)

                                    # find price box, strips away the tag info and leaves the price. If price isn't listed, skip
                                    try:
                                        price=soupd_item['soup_cat'+str(pg_count)].find("span",{"class":"ProductPriceOrderBox"}).get_text(strip=True)
                                    except AttributeError:
                                        price='NaN'

                                    # find description for product
                                    description=soupd_item[item].find("span",{"id":"lblDetailDes"}).get_text(strip=True)

                                    prodinfo=[]
                                    prodinfo.append(itemtitle)
                                    prodinfo.append(description)
                                    prodinfo.append(price)
                                    df0=pd.DataFrame(prodinfo)
                                    df0=df0.transpose()
                                    df0.columns=['title','description','price']
                                    df0.index=['Value']


                                    # find specification table
                                    spectable=soupd_item[item].find("table",{"class":"seoSpecTable1"})

                                    # loop through spec table and grab all attributes
                                    specrows=spectable.find_all('tr')

                                    # improved method to grab columns within each row
                                    list_rows=[]
                                    for row in specrows:
                                        cells=row.find_all('td')
                                        str_cells=str(cells)
                                        clean=re.compile('<.*?>')
                                        clean2=(re.sub(clean,'',str_cells))
                                        list_rows.append(clean2)
                                    type(clean2)

                                    # pass array to a dataframe
                                    df1=pd.DataFrame(list_rows)

                                    # use split function to separate headers from values
                                    df1_1=df1[0].str.split(',',n=1,expand=True)
                                    df1_1.columns=['Name','Value']

                                    # Clean up data
                                    df1_1['Name']=df1_1['Name'].str.replace('[','')
                                    df1_1['Name']=df1_1['Name'].str.replace(':','')
                                    df1_1['Value']=df1_1['Value'].str.replace(']','')

                                    # transpose dataframe because the headers are on the rows
                                    df1_1=df1_1.transpose()

                                    # replace headers with first row and delete first row
                                    df1_2=df1_1.rename(columns=df1_1.iloc[0])
                                    df1_3=df1_2.drop(df1_2.index[0])

                                    # find second specification table
                                    spectable2=soupd_item[item].find("table",{"class":"seoSpecTable2"})

                                    # loop through spec table and grab all attributes
                                    specrows2=spectable2.find_all('tr')

                                    # improved method to grab columns within each row
                                    list_rows2=[]
                                    for row in specrows2:
                                        cells=row.find_all('td')
                                        str_cells=str(cells)
                                        clean=re.compile('<.*?>')
                                        clean2=(re.sub(clean,'',str_cells))
                                        list_rows2.append(clean2)
                                    type(clean2)

                                    # pass array to a dataframe
                                    df2=pd.DataFrame(list_rows2)

                                    # use split function to separate headers from values
                                    df2_1=df2[0].str.split(',',n=1,expand=True)
                                    df2_1.columns=['Name','Value']

                                    # Clean up data
                                    df2_1['Name']=df2_1['Name'].str.replace('[','')
                                    df2_1['Name']=df2_1['Name'].str.replace(':','')
                                    df2_1['Value']=df2_1['Value'].str.replace(']','')

                                    # transpose dataframe because the headers are on the rows
                                    df2_1=df2_1.transpose()

                                    # replace headers with first row and delete first row
                                    df2_2=df2_1.rename(columns=df2_1.iloc[0])
                                    df2_3=df2_2.drop(df2_2.index[0])

                                    # merge this dataframe with the other
                                    dfmer=df1_3.join(df2_3,sort=True)
                                    dfmer=df0.join(dfmer,sort=True)
                                    dfmer['subsubcategory']=subsubtitle
                                    dfitem=dfitem.append(dfmer,sort=True)
                                    pg_count+=1
                                #==================================================================================
                                
                                # append each subsubcategory data
                                dfsubsub['subcategory']=subtitle
                                dfsubsub=dfsubsub.append(dfitem,sort=True)
                                cat_count_subsub+=1
                                subsubcount+=1
                            
                            print("There are",cat_count_subsub,"sub sub categories within this subcategory")  
                                        # So every other entry in the dictionary is the html page

                        except AttributeError:
                            print("There are no more descendents in this subcategory")
                
                    # append each subcategory to dataframe
                    dfsub['category']=title 
                    dfsub=dfsub.append(dfsubsub,sort=True)

                    cat_count_sub+=1
                    subcount+=1

                print("There are",cat_count_sub,"sub categories within this category")  
        else:
            cat_count+=1
            print("This category contains only drivers and will be skipped")
    else: 
        cat_count+=1
        print("This category contains only ballasts and will be skipped")
    
    # append each category dataframe
    #df=df.append(dfsub,sort=True)
    #df['group']=title    
    
    cat_count+=1  
    print("subcategory successfully scraped")

#reindex
#dfsub.index=np.arange(1,dfsub.category.count()+1,1) 
    
print("There are",cat_count,"categories")  
print("There are",items,"total items")
print("scraping success!")

> 
	Ballasts - Fluorescent - Platt Electric Supply

This category contains only ballasts and will be skipped
subcategory successfully scraped
> 
	Ballasts - HID - Platt Electric Supply

This category contains only ballasts and will be skipped
subcategory successfully scraped
> 
	Drivers - LED - Platt Electric Supply

This category contains only drivers and will be skipped
subcategory successfully scraped
> 
	Bulbs - Fluorescent - Platt Electric Supply

>> 
	Fluorescent - Circular - Platt Electric Supply

>>> 
	Circular Lamps - T5 - Platt Electric Supply

There are 14 items on page 1
There are 1 page(s) in this descendent
There are 14 total items
There is only one page
>>> 
	Circular Lamps - T6 - Platt Electric Supply

There are 3 items on page 1
There are 1 page(s) in this descendent
There are 3 total items
There is only one page
>>> 
	Circular Lamps - T9 - Platt Electric Supply

There are 15 items on page 1
There are: 0 items on page 2
There are 2 page(s) in this descendent
There are 

In [ ]:
# =====|| For Implementation ||=====

# Define dictionaries to hold soup and path info

pathd_cat={}
soupd_cat={}
beg_path='https://www.platt.com'
cat_count=0
subcount=0
cat_names=[]
items=0

# Grab website information
path='https://www.platt.com/platt-electric-supply/Bulbs-and-Ballasts/search.aspx?SectionID=10'
html=ureq.urlopen(path)
soup=BeautifulSoup(html,'html.parser')
type(soup)

# Grab links for all subcategories
linktable_cat=soup.find("div",{"class":"SubCategoryList"})
links_cat=linktable_cat.find_all('a')

# loop through all subcategories
for link in links_cat:
       
    # open subcategory    
    #print(link.get('href'))
    pathd_cat["path"+str(cat_count)]=link.get('href')
    tempath=beg_path+pathd_cat["path"+str(cat_count)]
    html=ureq.urlopen(tempath)

    # Grab subcategory page information and put in dictionary
    soupd_cat["soup_cat"+str(cat_count)]=BeautifulSoup(html,'html.parser')
    type(soupd_cat["soup_cat"+str(cat_count)])
    title=soupd_cat["soup_cat"+str(cat_count)].title.get_text()
    print(">",title)

    # define dictionaries and repeat for subcategories
    pathd_sub={}
    soupd_sub={}
    subtitles=[]

    title_sub = soupd_cat["soup_cat"+str(cat_count)].title.get_text()
    if "Ballasts" not in title_sub:
        if "Drivers" not in title_sub:
            if "Fluorescent" in title_sub: # test statement so i don't have to wait ages
                linktable_sub=soupd_cat["soup_cat"+str(cat_count)].find("div",{"class":"SubCategoryList"})
                links_sub=linktable_sub.find_all('a')

                cat_count_sub=0
                dfsub=pd.DataFrame() # define dataframe to append all dfsubsub's to
                
                for link in links_sub:

                    pathd_sub["path"+str(subcount)]=link.get('href')
                    tempath=beg_path+pathd_sub["path"+str(subcount)]
                    html=ureq.urlopen(tempath)


                    soupd_sub["soup_cat"+str(subcount)]=BeautifulSoup(html,'html.parser')
                    type(soupd_sub["soup_cat"+str(subcount)])

                    subtitle=soupd_sub["soup_cat"+str(subcount)].title.get_text()
                    if "Compact" in subtitle:
                        subtitles.append(subtitle)
                        print(">>",subtitle)

                        # define subsubcategory dictionaries and repeat again
                        pathd_subsub={}
                        soupd_subsub={}
                        subsubcount=0
                        subsubtitles=[]

                        # if there is no descendent, skip it
                        linktable_subsub=soupd_sub["soup_cat"+str(subcount)].find("div",{"class":"SubCategoryList"})

                        try:
                            links_subsub=linktable_subsub.find_all('a')
                            cat_count_subsub=0
                            
                            dfsubsub=pd.DataFrame() # define dataframe to append all dfitems to

                            for link in links_subsub:

                                #print(link.get('href'))
                                pathd_subsub["path"+str(subsubcount)]=link.get('href')
                                tempath=beg_path+pathd_subsub["path"+str(subsubcount)]
                                html=ureq.urlopen(tempath)


                                soupd_subsub["soup_cat"+str(subsubcount)]=BeautifulSoup(html,'html.parser')
                                type(soupd_subsub["soup_cat"+str(subsubcount)])

                                subsubtitle=soupd_subsub["soup_cat"+str(subsubcount)].title.get_text()
                            
                                subsubtitles.append(subsubtitle)
                                print(">>>",subsubtitle)


                                #==================================================================================
                                pathd_pagination={}
                                soupd_pagination={}
                                page_count=1

                                pathd_item={}
                                soupd_item={}
                                item_count=0

                                # grab last link in pagination, and create format for page url population
                                linktable_pagination=soupd_subsub["soup_cat"+str(subsubcount)].find("div",{"class":"searchTopBarPagination"})
                                links_page=linktable_pagination.find_all('a')
                                lastlink=linktable_pagination.find('a',{"class":"pg-arrow-last"})

                                # account for pages where there may not be pagination
                                try:
                                    lastpage=lastlink.get('href')
                                    linkformat=beg_path+lastpage
                                    last_pagenum=linkformat[-6]

                                    # loop through the range of total pages and create urls, starting at 2 because we have already got the first page
                                    pagination_urls=list()

                                    for i in range(2,int(last_pagenum)+1):

                                        # replace page values
                                        url_new=linkformat[:int(-6)]+str(i)+linkformat[int(-6)+1:]
                                        pagination_urls.append(url_new)
                                    lock=0

                                except AttributeError:
                                    lock=1

                                # look at first item page
                                linktable_item=soupd_subsub["soup_cat"+str(subsubcount)].find("div",{"class":"SearchProductsTable"})
                                links_item=linktable_item.find_all('a',{"class":"productListCatNumLink"})
                                for link in links_item:
                                    pathd_item["path"+str(item_count)]=link.get('href')

                                    tempath=beg_path+pathd_item["path"+str(item_count)]
                                    html=ureq.urlopen(tempath)

                                    soupd_item["soup_cat"+str(item_count)]=BeautifulSoup(html,'html.parser')
                                    type(soupd_item["soup_cat"+str(item_count)])
                                    itemtitle=soupd_item["soup_cat"+str(item_count)].title.get_text()
                                    #print(testtitle)
                                    item_count+=1
                                    items+=1

                                print("There are",item_count,"items on page 1")
                                page1_cnt=item_count

                                pgitem_count=0    


                                # loop through remaining pages if there are any

                                if lock==0:
                                    for page in pagination_urls:

                                        # get soup for that page
                                        html=ureq.urlopen(page)
                                        soup=BeautifulSoup(html,'html.parser')
                                        type(soup)

                                        # grab item page links on pagination page
                                        linktable_item=soup.find("div",{"class":"SearchProductsTable"})
                                        links=linktable_item.find_all('a',{"class":"productListCatNumLink"})

                                        pgitem_count=0
                                        for link in links:
                                            pathd_item["path"+str(item_count)]=link.get('href')

                                            tempath=beg_path+pathd_item["path"+str(item_count)]
                                            html=ureq.urlopen(tempath)

                                            soupd_item["soup_cat"+str(item_count)]=BeautifulSoup(html,'html.parser')
                                            type(soupd_item["soup_cat"+str(item_count)])
                                            itemtitle=soupd_item["soup_cat"+str(item_count)].title.get_text()
                                            #print(itemtitle)
                                            item_count+=1
                                            pgitem_count+=1
                                        page_count+=1

                                        print("There are:",pgitem_count,"items on page",page_count)
                                print("There are",page_count,"page(s) in this descendent")
                                print("There are",item_count,"total items")
                                if lock==1:
                                    print("There is only one page")

                                # ===== DATAFRAME INFORMATION =====
                                # With soups for all item pages, iterate through soups to gather information, then create dataframe from them.

                                # define dataframe to hold item page info
                                dfitem=pd.DataFrame() #********ISSUE: DF REDEFINED EVERY LOOP
                                pg_count=0

                                # loop through item pages
                                for item in soupd_item:
                                    itemtitle=soupd_item['soup_cat'+str(pg_count)].title.get_text()
                                    #print(itemtitle)

                                    # find price box, strips away the tag info and leaves the price. If price isn't listed, skip
                                    try:
                                        price=soupd_item['soup_cat'+str(pg_count)].find("span",{"class":"ProductPriceOrderBox"}).get_text(strip=True)
                                    except AttributeError:
                                        price='NaN'

                                    # find description for product
                                    description=soupd_item[item].find("span",{"id":"lblDetailDes"}).get_text(strip=True)

                                    prodinfo=[]
                                    prodinfo.append(itemtitle)
                                    prodinfo.append(description)
                                    prodinfo.append(price)
                                    df0=pd.DataFrame(prodinfo)
                                    df0=df0.transpose()
                                    df0.columns=['title','description','price']
                                    df0.index=['Value']


                                    # find specification table
                                    spectable=soupd_item[item].find("table",{"class":"seoSpecTable1"})

                                    # loop through spec table and grab all attributes
                                    specrows=spectable.find_all('tr')

                                    # improved method to grab columns within each row
                                    list_rows=[]
                                    for row in specrows:
                                        cells=row.find_all('td')
                                        str_cells=str(cells)
                                        clean=re.compile('<.*?>')
                                        clean2=(re.sub(clean,'',str_cells))
                                        list_rows.append(clean2)
                                    type(clean2)

                                    # pass array to a dataframe
                                    df1=pd.DataFrame(list_rows)

                                    # use split function to separate headers from values
                                    df1_1=df1[0].str.split(',',n=1,expand=True)
                                    df1_1.columns=['Name','Value']

                                    # Clean up data
                                    df1_1['Name']=df1_1['Name'].str.replace('[','')
                                    df1_1['Name']=df1_1['Name'].str.replace(':','')
                                    df1_1['Value']=df1_1['Value'].str.replace(']','')

                                    # transpose dataframe because the headers are on the rows
                                    df1_1=df1_1.transpose()

                                    # replace headers with first row and delete first row
                                    df1_2=df1_1.rename(columns=df1_1.iloc[0])
                                    df1_3=df1_2.drop(df1_2.index[0])

                                    # find second specification table
                                    spectable2=soupd_item[item].find("table",{"class":"seoSpecTable2"})

                                    # loop through spec table and grab all attributes
                                    specrows2=spectable2.find_all('tr')

                                    # improved method to grab columns within each row
                                    list_rows2=[]
                                    for row in specrows2:
                                        cells=row.find_all('td')
                                        str_cells=str(cells)
                                        clean=re.compile('<.*?>')
                                        clean2=(re.sub(clean,'',str_cells))
                                        list_rows2.append(clean2)
                                    type(clean2)

                                    # pass array to a dataframe
                                    df2=pd.DataFrame(list_rows2)

                                    # use split function to separate headers from values
                                    df2_1=df2[0].str.split(',',n=1,expand=True)
                                    df2_1.columns=['Name','Value']

                                    # Clean up data
                                    df2_1['Name']=df2_1['Name'].str.replace('[','')
                                    df2_1['Name']=df2_1['Name'].str.replace(':','')
                                    df2_1['Value']=df2_1['Value'].str.replace(']','')

                                    # transpose dataframe because the headers are on the rows
                                    df2_1=df2_1.transpose()

                                    # replace headers with first row and delete first row
                                    df2_2=df2_1.rename(columns=df2_1.iloc[0])
                                    df2_3=df2_2.drop(df2_2.index[0])

                                    # merge this dataframe with the other
                                    dfmer=df1_3.join(df2_3,sort=True)
                                    dfmer=df0.join(dfmer,sort=True)
                                    dfmer['subsubcategory']=subsubtitle
                                    dfitem=dfitem.append(dfmer,sort=True)
                                    pg_count+=1
                                #==================================================================================
                                
                                # append each subsubcategory data
                                dfsubsub=dfsubsub.append(dfitem,sort=True)
                                dfsubsub['subcategory']=subtitle
                                cat_count_subsub+=1
                                subsubcount+=1
                            
                            print("There are",cat_count_subsub,"sub sub categories within this subcategory")  
                                        # So every other entry in the dictionary is the html page

                        except AttributeError:
                            print("There are no more descendents in this subcategory")

                print("There are",subsubcount,"subsub categories within this sub category")
                print("success")
                
                # append each subsubcategory data
                dfsub=dfsub.append(dfsubsub,sort=True)
                dfsub['category']=title    
                cat_count_sub+=1
                subcount+=1

                print("There are",cat_count_sub,"sub categories within this category")  
        else:
            cat_count+=1
            print("This category contains only drivers and will be skipped")
    else: 
        cat_count+=1
        print("This category contains only ballasts and will be skipped")
    cat_count+=1  
    print("subcategory successfully scraped")

    
print("There are",cat_count,"categories")  
print("There are",items,"total items")
print("scraping success!")

## Testing

In [3]:
dfsub.describe()

,Amps,Base,Cat,Category,Color,Color Rendering Index (CRI),Color Temperature,Country of Origin,Diameter,Family / Style,...,Type,UPC,Volts AC,Wattage,category,description,price,subcategory,subsubcategory,title
count,1,285,318,308,107,281,295,52,7,3,...,302,318,157,301,300,318,318,288,318,318
unique,1,33,271,18,4,13,14,3,5,2,...,35,249,4,45,1,271,134,4,17,271
top,23,2-Pin (G13),F96T8/ADV850/XEW/ALTO 51W,,White,82,4100 (Cool White),UNITED STATES,2.200 in,Pentron,...,"Compact Fluorescent, Lamp",Not Available,120,32,\r\n\tBulbs - Fluorescent - Platt Electric Sup...,"Fluorescent, High Lumen, Advantage Lamp, 32W, ...",NaN,\r\n\tFluorescent - Compact - Platt Electric S...,\r\n\tT8 - Long Life - Platt Electric Supply\r\n,"\r\n\tPhilips Lighting - F32T8/TL935/ALTO, T8 ..."
freq,1,91,2,90,38,176,66,49,2,2,...,80,23,154,64,300,2,24,170,76,2


In [4]:
dfsub.head(2)

,Amps,Base,Cat,Category,Color,Color Rendering Index (CRI),Color Temperature,Country of Origin,Diameter,Family / Style,...,Type,UPC,Volts AC,Wattage,category,description,price,subcategory,subsubcategory,title
Value,NaN,4-Pin (2GX13),TL5C 40W/835,Circular Lamps - T5,NaN,85,3500 (Warm White),NaN,NaN,NaN,...,"Lamp, Fluorescent",Not Available,NaN,40,\r\n\tBulbs - Fluorescent - Platt Electric Sup...,"Fluorescent, Circline Lamp, Programmed Start, ...",$24.36,\r\n\tFluorescent - Circular - Platt Electric ...,\r\n\tCircular Lamps - T5 - Platt Electric Sup...,"\r\n\tPhilips Lighting - TL5C 40W/835, Circula..."
Value,NaN,4-Pin (2GX13),TL5C 22W/830,Circular Lamps - T5,Warm White,85,3000 (Warm White),NaN,NaN,NaN,...,"Lamp, Fluorescent",Not Available,NaN,22,\r\n\tBulbs - Fluorescent - Platt Electric Sup...,"Fluorescent, Circline Lamp, Programmed Start, ...",$24.36,\r\n\tFluorescent - Circular - Platt Electric ...,\r\n\tCircular Lamps - T5 - Platt Electric Sup...,"\r\n\tPhilips Lighting - TL5C 22W/830, Circula..."


In [7]:
dfsub.subsubcategory.unique()

array(['\r\n\tCircular Lamps - T5 - Platt Electric Supply\r\n',
       '\r\n\tCircular Lamps - T6 - Platt Electric Supply\r\n',
       '\r\n\tCircular Lamps - T9 - Platt Electric Supply\r\n',
       '\r\n\tCFL - 2-Pin Base - Platt Electric Supply\r\n',
       '\r\n\tCFL - 4-Pin Base - Platt Electric Supply\r\n',
       '\r\n\tCFL - Square - Platt Electric Supply\r\n',
       '\r\n\tCFL - Twist Lock Base - Platt Electric Supply\r\n',
       '\r\n\tCFL - Screw-In - Decorative - Platt Electric Supply\r\n',
       '\r\n\tCFL - Screw-In - Reflector - Platt Electric Supply\r\n',
       '\r\n\tCFL - Screw-In - Traditional - Platt Electric Supply\r\n',
       '\r\n\tCFL - Replacement 2-Piece Top - Platt Electric Supply\r\n',
       '\r\n\tT8 - Colored - Platt Electric Supply\r\n',
       '\r\n\tT8 - High Output - Platt Electric Supply\r\n',
       '\r\n\tT8 - Long Life - Platt Electric Supply\r\n',
       '\r\n\tT8 - Preheat - Platt Electric Supply\r\n',
       '\r\n\tU-Bent Lamps - T8 - Platt

In [12]:
dfsub.columns

Index(['Amps', 'Base', 'Cat', 'Category ', 'Color',
       'Color Rendering Index (CRI)', 'Color Temperature', 'Country of Origin',
       'Diameter', 'Family / Style', 'Length', 'Lumens', 'Material', 'Mfr',
       'Platt Cat', 'Platt Item', 'Shape', 'Type', 'UPC', 'Volts AC',
       'Wattage', 'description', 'price', 'subcategory', 'subsubcategory',
       'title'],
      dtype='object')

In [9]:
dfsub.count()

Amps                             2
Base                           242
Cat                            271
Category                       263
Color                          109
Color Rendering Index (CRI)    232
Color Temperature              246
Country of Origin                6
Diameter                        13
Family / Style                   7
Length                          41
Lumens                         256
Material                        33
Mfr                              8
Platt Cat                      271
Platt Item                     271
Shape                          235
Type                           249
UPC                            271
Volts AC                       174
Wattage                        258
description                    271
price                          271
subcategory                    252
subsubcategory                 271
title                          271
dtype: int64

In [7]:
dfsub.dtypes

Amps                           object
Base                           object
Cat                            object
Category                       object
Color                          object
Color Rendering Index (CRI)    object
Color Temperature              object
Country of Origin              object
Diameter                       object
Family / Style                 object
Length                         object
Lumens                         object
Material                       object
Mfr                            object
Platt Cat                      object
Platt Item                     object
Shape                          object
Type                           object
UPC                            object
Volts AC                       object
Wattage                        object
description                    object
price                          object
subcategory                    object
subsubcategory                 object
title                          object
category    

In [12]:
dfsub.columns

Index(['Amps', 'Base', 'Cat', 'Category ', 'Color',
       'Color Rendering Index (CRI)', 'Color Temperature', 'Country of Origin',
       'Diameter', 'Family / Style', 'Length', 'Lumens', 'Material', 'Mfr',
       'Platt Cat', 'Platt Item', 'Shape', 'Type', 'UPC', 'Volts AC',
       'Wattage', 'description', 'price', 'subcategory', 'subsubcategory',
       'title', 'category'],
      dtype='object')

# Function Based Scraper

In [8]:
### category->subcategory->subsubcategory->item page->pagination->dataframe
# define dataframe to hold data

df_web=pd.DataFrame()
def webscrape(path,dataframe,export=False,savepath='none'):

    # Define dictionaries to hold soup and path info
    pathd_cat={}
    soupd_cat={}
    beg_path='https://www.platt.com'
    cat_count=0
    subcount=0
    cat_names=[]

    # open website and gather information
    html=ureq.urlopen(path)
    soup=BeautifulSoup(html,'html.parser')
    type(soup)

    # gather links
    linktable_cat=soup.find("div",{"class":"SubCategoryList"})
    links_cat=linktable_cat.find_all('a')

    for link in links_cat:

        #print(link.get('href'))
        pathd_cat["path"+str(cat_count)]=link.get('href')
        tempath=beg_path+pathd_cat["path"+str(cat_count)]
        html=ureq.urlopen(tempath)


        soupd_cat["soup_cat"+str(cat_count)]=BeautifulSoup(html,'html.parser')
        type(soupd_cat["soup_cat"+str(cat_count)])
        title=soupd_cat["soup_cat"+str(cat_count)].title.get_text()
        print(">",title)

        pathd_sub={}
        soupd_sub={}
        subtitles=[]

        title_sub = soupd_cat["soup_cat"+str(cat_count)].title.get_text()
        if "Ballasts" not in title_sub:
            if "Drivers" not in title_sub:
                if "Fluorescent" in title_sub: # test statement so i don't have to wait ages
                    linktable_sub=soupd_cat["soup_cat"+str(cat_count)].find("div",{"class":"SubCategoryList"})
                    links_sub=linktable_sub.find_all('a')

                    cat_count_sub=0
                    dataframe=pd.DataFrame() # define dataframe to append all dfsubsub's to

                    for link in links_sub:

                        pathd_sub["path"+str(subcount)]=link.get('href')
                        tempath=beg_path+pathd_sub["path"+str(subcount)]
                        html=ureq.urlopen(tempath)


                        soupd_sub["soup_cat"+str(subcount)]=BeautifulSoup(html,'html.parser')
                        type(soupd_sub["soup_cat"+str(subcount)])

                        subtitle=soupd_sub["soup_cat"+str(subcount)].title.get_text()
                        if "Compact" in subtitle:
                            subtitles.append(subtitle)
                            print(">>",subtitle)

                            #subsubcategories
                            pathd_subsub={}
                            soupd_subsub={}
                            subsubcount=0
                            subsubtitles=[]

                            # if there is no descendent, skip it
                            linktable_subsub=soupd_sub["soup_cat"+str(subcount)].find("div",{"class":"SubCategoryList"})

                            try:
                                links_subsub=linktable_subsub.find_all('a')
                                cat_count_subsub=0

                                dfsubsub=pd.DataFrame() # define dataframe to append all dfitems to

                                for link in links_subsub:

                                    #print(link.get('href'))
                                    pathd_subsub["path"+str(subsubcount)]=link.get('href')
                                    tempath=beg_path+pathd_subsub["path"+str(subsubcount)]
                                    html=ureq.urlopen(tempath)


                                    soupd_subsub["soup_cat"+str(subsubcount)]=BeautifulSoup(html,'html.parser')
                                    type(soupd_subsub["soup_cat"+str(subsubcount)])

                                    subsubtitle=soupd_subsub["soup_cat"+str(subsubcount)].title.get_text()

                                    subsubtitles.append(subsubtitle)
                                    print(">>>",subsubtitle)


                                    #==================================================================================
                                    pathd_pagination={}
                                    soupd_pagination={}
                                    page_count=1

                                    pathd_item={}
                                    soupd_item={}
                                    item_count=0

                                    # grab last link in pagination, and create format for page url population
                                    linktable_pagination=soupd_subsub["soup_cat"+str(subsubcount)].find("div",{"class":"searchTopBarPagination"})
                                    links_page=linktable_pagination.find_all('a')
                                    lastlink=linktable_pagination.find('a',{"class":"pg-arrow-last"})

                                    # account for pages where there may not be pagination
                                    try:
                                        lastpage=lastlink.get('href')
                                        linkformat=beg_path+lastpage
                                        last_pagenum=linkformat[-6]

                                        # loop through the range of total pages and create urls, starting at 2 because we have already got the first page
                                        pagination_urls=list()

                                        for i in range(2,int(last_pagenum)+1):

                                            # replace page values
                                            url_new=linkformat[:int(-6)]+str(i)+linkformat[int(-6)+1:]
                                            pagination_urls.append(url_new)
                                        lock=0

                                    except AttributeError:
                                        lock=1

                                    # look at first item page
                                    linktable_item=soupd_subsub["soup_cat"+str(subsubcount)].find("div",{"class":"SearchProductsTable"})
                                    links_item=linktable_item.find_all('a',{"class":"productListCatNumLink"})
                                    for link in links_item:
                                        pathd_item["path"+str(item_count)]=link.get('href')

                                        tempath=beg_path+pathd_item["path"+str(item_count)]
                                        html=ureq.urlopen(tempath)

                                        soupd_item["soup_cat"+str(item_count)]=BeautifulSoup(html,'html.parser')
                                        type(soupd_item["soup_cat"+str(item_count)])
                                        itemtitle=soupd_item["soup_cat"+str(item_count)].title.get_text()
                                        #print(testtitle)
                                        item_count+=1

                                    print("There are",item_count,"items on page 1")
                                    page1_cnt=item_count

                                    pgitem_count=0    


                                    # loop through remaining pages if there are any

                                    if lock==0:
                                        for page in pagination_urls:

                                            # get soup for that page
                                            html=ureq.urlopen(page)
                                            soup=BeautifulSoup(html,'html.parser')
                                            type(soup)

                                            # grab item page links on pagination page
                                            linktable_item=soup.find("div",{"class":"SearchProductsTable"})
                                            links=linktable_item.find_all('a',{"class":"productListCatNumLink"})

                                            pgitem_count=0
                                            for link in links:
                                                pathd_item["path"+str(item_count)]=link.get('href')

                                                tempath=beg_path+pathd_item["path"+str(item_count)]
                                                html=ureq.urlopen(tempath)

                                                soupd_item["soup_cat"+str(item_count)]=BeautifulSoup(html,'html.parser')
                                                type(soupd_item["soup_cat"+str(item_count)])
                                                itemtitle=soupd_item["soup_cat"+str(item_count)].title.get_text()
                                                #print(itemtitle)
                                                item_count+=1
                                                pgitem_count+=1
                                            page_count+=1

                                            print("There are:",pgitem_count,"items on page",page_count)
                                    print("There are",page_count,"page(s) in this descendent")
                                    print("There are",item_count,"total items")
                                    if lock==1:
                                        print("There is only one page")

                                    # ===== DATAFRAME INFORMATION =====
                                    # With soups for all item pages, iterate through soups to gather information, then create dataframe from them.

                                    # define dataframe to hold item page info
                                    dfitem=pd.DataFrame() #********ISSUE: DF REDEFINED EVERY LOOP
                                    pg_count=0

                                    # loop through item pages
                                    for item in soupd_item:
                                        itemtitle=soupd_item['soup_cat'+str(pg_count)].title.get_text()
                                        #print(itemtitle)

                                        # find price box, strips away the tag info and leaves the price. If price isn't listed, skip
                                        try:
                                            price=soupd_item['soup_cat'+str(pg_count)].find("span",{"class":"ProductPriceOrderBox"}).get_text(strip=True)
                                        except AttributeError:
                                            price='NaN'

                                        # find description for product
                                        description=soupd_item[item].find("span",{"id":"lblDetailDes"}).get_text(strip=True)

                                        prodinfo=[]
                                        prodinfo.append(itemtitle)
                                        prodinfo.append(description)
                                        prodinfo.append(price)
                                        df0=pd.DataFrame(prodinfo)
                                        df0=df0.transpose()
                                        df0.columns=['title','description','price']
                                        df0.index=['Value']


                                        # find specification table
                                        spectable=soupd_item[item].find("table",{"class":"seoSpecTable1"})

                                        # loop through spec table and grab all attributes
                                        specrows=spectable.find_all('tr')

                                        # improved method to grab columns within each row
                                        list_rows=[]
                                        for row in specrows:
                                            cells=row.find_all('td')
                                            str_cells=str(cells)
                                            clean=re.compile('<.*?>')
                                            clean2=(re.sub(clean,'',str_cells))
                                            list_rows.append(clean2)
                                        type(clean2)

                                        # pass array to a dataframe
                                        df1=pd.DataFrame(list_rows)

                                        # use split function to separate headers from values
                                        df1_1=df1[0].str.split(',',n=1,expand=True)
                                        df1_1.columns=['Name','Value']

                                        # Clean up data
                                        df1_1['Name']=df1_1['Name'].str.replace('[','')
                                        df1_1['Name']=df1_1['Name'].str.replace(':','')
                                        df1_1['Value']=df1_1['Value'].str.replace(']','')

                                        # transpose dataframe because the headers are on the rows
                                        df1_1=df1_1.transpose()

                                        # replace headers with first row and delete first row
                                        df1_2=df1_1.rename(columns=df1_1.iloc[0])
                                        df1_3=df1_2.drop(df1_2.index[0])

                                        # find second specification table
                                        spectable2=soupd_item[item].find("table",{"class":"seoSpecTable2"})

                                        # loop through spec table and grab all attributes
                                        specrows2=spectable2.find_all('tr')

                                        # improved method to grab columns within each row
                                        list_rows2=[]
                                        for row in specrows2:
                                            cells=row.find_all('td')
                                            str_cells=str(cells)
                                            clean=re.compile('<.*?>')
                                            clean2=(re.sub(clean,'',str_cells))
                                            list_rows2.append(clean2)
                                        type(clean2)

                                        # pass array to a dataframe
                                        df2=pd.DataFrame(list_rows2)

                                        # use split function to separate headers from values
                                        df2_1=df2[0].str.split(',',n=1,expand=True)
                                        df2_1.columns=['Name','Value']

                                        # Clean up data
                                        df2_1['Name']=df2_1['Name'].str.replace('[','')
                                        df2_1['Name']=df2_1['Name'].str.replace(':','')
                                        df2_1['Value']=df2_1['Value'].str.replace(']','')

                                        # transpose dataframe because the headers are on the rows
                                        df2_1=df2_1.transpose()

                                        # replace headers with first row and delete first row
                                        df2_2=df2_1.rename(columns=df2_1.iloc[0])
                                        df2_3=df2_2.drop(df2_2.index[0])

                                        # merge this dataframe with the other
                                        dfmer=df1_3.join(df2_3,sort=True)
                                        dfmer=df0.join(dfmer,sort=True)
                                        dfmer['subsubcategory']=subsubtitle
                                        dfitem=dfitem.append(dfmer,sort=True)
                                        pg_count+=1
                                    #==================================================================================

                                    # append each subcategory data
                                    dfsubsub=dfsubsub.append(dfitem,sort=True)
                                    dfsubsub['subcategory']=subtitle
                                    cat_count_subsub+=1
                                    subsubcount+=1

                                print("There are",cat_count_subsub,"sub sub categories within this subcategory")  
                                            # So every other entry in the dictionary is the html page

                            except AttributeError:
                                print("There are no more descendents in this subcategory")

                    print("There are",subsubcount,"subsub categories within this sub category")
                    print("success")

                    # append each subsubcategory data
                    dataframe=dataframe.append(dfsubsub,sort=True)
                    dataframe['category']=title    
                    cat_count_sub+=1
                    subcount+=1

                    print("There are",cat_count_sub,"sub categories within this category")  
            else:
                cat_count+=1
                print("This category contains only drivers and will be skipped")
        else: 
            cat_count+=1
            print("This category contains only ballasts and will be skipped")
        cat_count+=1  
        print("subcategory successfully scraped")


    print("There are",cat_count,"categories")  
    print("scraping success!")
    
    return dataframe
    if export == True:
        dataframe.to_csv(savepath,encoding='utf-8')
        

In [9]:
path='https://www.platt.com/platt-electric-supply/Bulbs-and-Ballasts/search.aspx?SectionID=10' # testing website
savepath=r'C:\Users\apetersen\Nexant Market Segmentation\Exports\Market Segment Data.csv' # testing savepath
dftest=webscrape(path,df_web)

> 
	Ballasts - Fluorescent - Platt Electric Supply

This category contains only ballasts and will be skipped
subcategory successfully scraped
> 
	Ballasts - HID - Platt Electric Supply

This category contains only ballasts and will be skipped
subcategory successfully scraped
> 
	Drivers - LED - Platt Electric Supply

This category contains only drivers and will be skipped
subcategory successfully scraped
> 
	Bulbs - Fluorescent - Platt Electric Supply

>> 
	Fluorescent - Compact - Platt Electric Supply

>>> 
	CFL - 2-Pin Base - Platt Electric Supply

There are 15 items on page 1
There are: 15 items on page 2
There are: 15 items on page 3
There are: 15 items on page 4
There are 4 page(s) in this descendent
There are 60 total items
>>> 
	CFL - 4-Pin Base - Platt Electric Supply

There are 15 items on page 1
There are: 15 items on page 2
There are: 15 items on page 3
There are: 15 items on page 4
There are 4 page(s) in this descendent
There are 60 total items
>>> 
	CFL - Square - Platt El

In [7]:
dfsub

NameError: name 'dfsub' is not defined

In [12]:
dataframe

NameError: name 'dataframe' is not defined